In [1]:
import struct,sys
import h5py    
from pyhdf.SD import SD, SDC
import numpy as np    
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/Hao/mylibrary/FBL/')
import FBL_TOOL as ft

In [3]:
Tscale = 2.807067e7        # K
Bscale = 2.2068908         # G
Vscale = 481.3711          # km/s
rhoscale = 1.6726e-16      # g/cm^3
mh = 1.674e-24             # g/cm^3
ne2nh = 0.83

In [4]:
Rrange = np.array([20,120])

path = '/Volumes/Data/FBL/corona-2130-model1/'


items = ['br002', 'bt002','bp002', 'rho002', 't002', 'vr002', 'vt002', 'vp002']
dic = ['Br','Bt','Bp', 'rho', 'temp', 'Vr','Vt','Vp']

data = {}

hdf_obj = SD(path+items[0]+'.hdf', SDC.READ)

print(hdf_obj.datasets())


data['r'] = np.array(hdf_obj.select('fakeDim2'))[Rrange[0]:Rrange[1]]
data['theta'] = np.array(hdf_obj.select('fakeDim1'))
data['phi'] = np.array(hdf_obj.select('fakeDim0'))

data['Br'] = np.array(hdf_obj.select('Data-Set-2'))[:,:,Rrange[0]:Rrange[1]]

lr = len(data['r'])
ltheta = len(data['theta'])
lphi = len(data['phi'])


data['Bt'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['Bp'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['temp'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['rho'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['Vr'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['Vt'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)
data['Vp'] = np.zeros((lphi,ltheta,lr),dtype=np.float64)

for indx in range(1,len(items)):
  print(indx)
  hdf_obj = SD(path+items[indx]+'.hdf', SDC.READ)
    
  r = np.array(hdf_obj.select('fakeDim2'))
  theta = np.array(hdf_obj.select('fakeDim1'))
  phi = np.array(hdf_obj.select('fakeDim0'))

  llr = len(r)
  lltheta = len(theta)
  llphi = len(phi)
  
  data0 = np.array(hdf_obj.select('Data-Set-2'))

  for ir in range(lr):
    if data['r'][ir]<=r[0]:
      ir0 = 0
      Ratio_R = 0 
    elif data['r'][ir]>=r[llr-1]:
      ir0 = llr-2
      Ratio_R = 1.0
    else:
      for i in range(llr-1):
        if(data['r'][ir]>=r[i] and data['r'][ir]<=r[i+1]):
          ir0 = i;
          Ratio_R = (data['r'][ir]-r[i])/(r[i+1]-r[i]);
          break
        
    for itheta in range(ltheta):

      if data['theta'][itheta]<=theta[0]:
        itheta0 = 0
        Ratio_Theta = 0 
      elif data['theta'][itheta]>=theta[lltheta-1]:
        itheta0 = lltheta-2
        Ratio_Theta = 1.0
      else:
        for i in range(lltheta):
          if(data['theta'][itheta]>=theta[i] and data['theta'][itheta]<=theta[i+1]):
            itheta0 = i;
            Ratio_Theta = (data['theta'][itheta]-theta[i])/(theta[i+1]-theta[i]);
            break
          
      for iphi in range(lphi):
        if data['phi'][iphi]<=phi[0]:
          iphi0 = 0
          Ratio_Phi = 0 
        elif data['phi'][iphi]>=phi[llphi-1]:
          iphi0 = llphi-2
          Ratio_Phi = 1.0
        else:
          for i in range(len(phi)):
            if(data['phi'][iphi]>=phi[i] and data['phi'][iphi]<=phi[i+1]):
              iphi0 = i;
              Ratio_Phi = (data['phi'][iphi]-phi[i])/(phi[i+1]-phi[i]);
              break
          
        data[dic[indx]][iphi,itheta,ir] = ft.interp3d(data0, iphi0, itheta0, ir0, 
                                                 Ratio_Phi, Ratio_Theta, Ratio_R)



data['Br'] = data['Br']*Bscale
data['Bt'] = data['Bt']*Bscale
data['Bp'] = data['Bp']*Bscale
data['temp'] = data['temp']*Tscale
data['rho'] = data['rho']*rhoscale/mh/ne2nh
data['Vr'] = data['Vr']*Vscale
data['Vt'] = data['Vt']*Vscale
data['Vp'] = data['Vp']*Vscale

rhotype = 3
btype = 1
vtype = 1
path = '/Volumes/Data/FBL/'

filename = path+'model'
ft.writemodel(filename, data, rhotype=rhotype, \
              btype=btype, vtype=vtype)

{'fakeDim0': (('fakeDim0',), (181,), 5, 0), 'fakeDim1': (('fakeDim1',), (100,), 5, 1), 'fakeDim2': (('fakeDim2',), (151,), 5, 2), 'Data-Set-2': (('fakeDim0', 'fakeDim1', 'fakeDim2'), (181, 100, 151), 5, 3)}
1
2
3
4
5
6
7
R range =  1.0125712156295776 5.795185089111328


{'check': True}